In [1]:
import sys
sys.path.append("D:/trading")  # o el path donde está tu carpeta my_modules

In [2]:
import pandas as pd
import numpy as np
import itertools
from joblib import Parallel, delayed
from tqdm import tqdm

# Cargar históricos locales
import os
ruta_historicos = "D:/trading/data/historic"
historicos = {}
for archivo in os.listdir(ruta_historicos):
    if archivo.endswith(".parquet"):
        ticker = archivo.replace(".parquet", "")
        df = pd.read_parquet(os.path.join(ruta_historicos, archivo))
        historicos[ticker] = df
print(f"Símbolos cargados: {len(historicos)}")






Símbolos cargados: 48


In [3]:
param_grid = list(itertools.product(
    [True, False],              # confirmar_al_dia_siguiente
    [0.006, 0.008, 0.010]       # atr_threshold
))
print(f"Total combinaciones: {len(param_grid)}")




Total combinaciones: 6


In [4]:
from my_modules.estrategias.v4 import macd_hist_reversal_v4

def simular_combinacion(params):
    confirmar, atr_th = params
    resultados = []

    for ticker, df_precio in historicos.items():
        df_signals = macd_hist_reversal_v4.generar_senales(df_precio,
            confirmar_al_dia_siguiente=confirmar,
            atr_threshold=atr_th,
            debug=False
        )
        df_signals = df_signals[df_signals["signal"] != "hold"]
        if df_signals.empty:
            continue

        df_precio = df_precio.copy()
        df_precio = df_precio.merge(df_signals[["fecha", "signal"]], on="fecha")

        for _, row in df_precio.iterrows():
            fecha_entrada = row["fecha"]
            precio_entrada = row["close"]
            df_rango = df_precio[(df_precio["fecha"] > fecha_entrada) & (df_precio["fecha"] <= fecha_entrada + pd.Timedelta(days=7))]
            if df_rango.empty:
                continue

            tipo_salida = "TIMEOUT"
            fila_salida = df_rango.iloc[-1]

            for _, f in df_rango.iterrows():
                if row["signal"] == "buy":
                    if f["high"] >= precio_entrada * 1.05:
                        tipo_salida = "TP"; fila_salida = f; break
                    if f["low"] <= precio_entrada * 0.97:
                        tipo_salida = "SL"; fila_salida = f; break
                elif row["signal"] == "sell":
                    if f["low"] <= precio_entrada * 0.95:
                        tipo_salida = "TP"; fila_salida = f; break
                    if f["high"] >= precio_entrada * 1.03:
                        tipo_salida = "SL"; fila_salida = f; break

            precio_salida = fila_salida["close"]
            dias = (fila_salida["fecha"] - fecha_entrada).days
            resultado = precio_salida - precio_entrada if row["signal"] == "buy" else precio_entrada - precio_salida
            ret_pct = (precio_salida / precio_entrada - 1) * (1 if row["signal"] == "buy" else -1)
            log_ret = np.log(precio_salida / precio_entrada) * (1 if row["signal"] == "buy" else -1)

            resultados.append({
                "resultado": resultado,
                "resultado_pct": ret_pct,
                "log_ret": log_ret,
                "f_win": int(resultado > 0),
                "dias": dias
            })

    df = pd.DataFrame(resultados)
    if df.empty:
        return {
            "confirmar_al_dia_siguiente": confirmar,
            "atr_threshold": atr_th,
            "n_trades": 0,
            "winrate": 0,
            "avg_profit": 0,
            "score": -999
        }

    return {
        "confirmar_al_dia_siguiente": confirmar,
        "atr_threshold": atr_th,
        "n_trades": len(df),
        "winrate": df["f_win"].mean(),
        "avg_profit": df["resultado"].mean(),
        "score": df["resultado"].mean() * df["f_win"].mean()
    }

ImportError: cannot import name 'macd_hist_reversal_v4' from 'my_modules.estrategias.v4' (unknown location)

In [ ]:
from itertools import chain
from pandas import DataFrame

resultados = Parallel(n_jobs=-1)(
    delayed(simular_combinacion)(params) for params in tqdm(param_grid)
)

df_tuning = pd.DataFrame(resultados).sort_values("score", ascending=False).reset_index(drop=True)

print("Top combinaciones por score:")
display(df_tuning.head(10))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=df_tuning,
    x="winrate",
    y="avg_profit",
    size="n_trades",
    hue="score",
    palette="viridis",
    sizes=(50, 400),
    edgecolor="black",
    alpha=0.8
)
plt.axhline(0, color="gray", linestyle="--")
plt.axvline(0.5, color="gray", linestyle="--")
plt.title("Grid Search macd_hist_reversal_v4 — Winrate vs Profit")
plt.xlabel("Winrate")
plt.ylabel("Avg Profit ($)")
plt.tight_layout()
plt.show()